# Google Colab Setup

In [ ]:
#@title Setup Google Colab by running this cell only once (ignore this if run locally) {display-mode: "form"}
import sys

if 'google.colab' in sys.modules:
    # Clone GitHub repository
    !git clone https://github.com/epfl-exts/aiml2days.git
        
    # Copy files required to run the code
    !cp -r "aiml2days/notebooks/data" "aiml2days/notebooks/data_prep_tools.py" "aiml2days/notebooks/EDA_tools.py" "aiml2days/notebooks/modeling_tools.py" . 
    
    # Install packages via pip
    !pip install -r "aiml2days/colab-requirements.txt"
    
    # Restart Runtime
    import os
    os.kill(os.getpid(), 9)

# Build a spam detector

### Task 

We want to build a Spam detector which, given examples of spam emails (e.g. flagged by users) and examples of regular (non-spam, also called "ham") emails, learns how to flag new unseen emails as spam or non-spam.

### Data

In the previous notebooks we have covered the following steps:
* Load the data
* Text preprocessing
* Feature extraction
* Data exploration

The different feature sets can be loaded with the `load_feature_space()`-function. The options for the feature sets are:
* "num": numerical features
* "text": text features
* "num_text": numerical and text features combined
* "embedding": embedding features

### In this notebook

Our aim is to build a simple spam detector. We will start by exploring the different datasets, before we will build a simple spam detector and evaluate the model. We will use the following steps:
* Load the features
* Split the data into training and test set
* Train a simple spam detector
* Evaluate the model
* Analyze misclassified samples



## Load the data

In [ ]:
# Load libraries and helper functions
%run data_prep_tools.py
%run EDA_tools.py
%run modeling_tools.py

In [ ]:
# Load the data sets
num_features_df = load_feature_space("num", no_labels=False)
text_features_df = load_feature_space("text", no_labels=False)
num_text_features_df = load_feature_space("num_text", no_labels=False)
embeddings_df = load_feature_space("embedding", no_labels=False)
labels = load_labels()

Let's check the number of samples per class in the data.

In [ ]:
plot_class_frequency(labels)

<div class="alert alert-success">


<h3>Questions</h3>

Suppose you applied a **very naive approach** to the spam detection problem **that uses none of the features**: _You just either classify all emails as "spam" or as "non-spam"._

__Q1.__ How many emails would be classified correctly in each case?  

__Q2.__ Which approach would be more successful?

This naive kind of approach is useful to establish a **baseline** for the performance of our more complex classifiers.  
In this notebook we will build various spam detectors and compare their performance to this baseline.
</div>

#### Give your answer here:

1.    



2.







# Model building

We are now ready to build our machine learning model for detecting spams. 

Throughout this notebook we will use a **Logistic Regression classifier**. Here is why:
- It is a simple and efficient model for binary classification tasks. 
- It is a good baseline for more complex models. 
- It is fast to train and thus allows us to quickly iterate on our model and try out different settings.
- It is also easy to interpret and allows us to explore where our model makes mistakes.

Below you will
- build a first simple model.
- tune the main hyperparameter `C` for the model using a cross-validated grid search.
- explore different feature sets and see how they affect the performance of the model.
- explore the effect of different evaluation metrics.
- Explore misclassified samples.

### A first trial

As a first trial, we will use the `num` feature set with a simpl. The accuracy is defined as the number of correct predictions divided by the total number of predictions.  

In [ ]:
# Train/test splitting
df_train, df_test = train_test_split_(num_features_df)

# Fit model on the train data
model = fit_model(df_train, C=1)

# Print predictions on test set
plot_confusion_matrix(df_test, model)

<div class="alert alert-success">


<h3>Questions</h3>

Let's explore the plot above. 

__Q1.__ Which numbers tell use the correct predictions for each class?

__Q2.__ Which numbers tell use the failed predictions for each class?

__Q3.__ What class faired better?


</div>

#### Give your answer here:

1.    




2.




3.







The classification report provides us with different 4 metrics to evaluate the performance of our model: 3 metrics for each class and the overall accuracy. 

The **accuracy** is the number of correct predictions divided by the total number of predictions.

The **precision** is the number of true positives divided by the number of true positives plus the number of false positives.

The **recall** is the number of true positives divided by the number of true positives plus the number of false negatives.

The **f1-score** is the harmonic mean of precision and recall.

In [ ]:
# Print classification report for test set
classification_report_(df_test, model)

You can consider changing the `C` parameter in the `fit_model` function to see how it affects the model performance. But this is seen as bad practice be cause you are tuning the mmodel to the test set, which you also use for evaluation.

Instead let's use a more appropriate approach below.

### A more systematic approach to fine tuning

We will use 5-fold cross-validation. So the validation sets are automatically created internally. The test set will be used to `evaluate` the performance of our model. The process will automatically choose the best model for us. We also collect all the results from the cross-validation so we can plot them below.

In [ ]:
%run modeling_tools.py

In [ ]:
# Train/test splitting
df_train, df_test = train_test_split_(num_features_df)
# text_features_df takes 45mins

# Fit model on the train data
model, cv_results = fit_log_reg_model(df_train)

#### Tracking overfitting

Below we plot the results of the cross-validation. The x-axis shows the different values of `C` we tried, and the y-axis shows the metric of the model evaluated on the training set (blue) and on the validation set (orange). The red cross shows the value of the best `C` we found.

We are interested in the gap between the training and validation curves. If the gap is small, it means that our model is not overfitting and generalizes well to unseen data. If the gap is large, it means that our model is overfitting. This indicates that the model has learned irrelevant information like noise that does not reflect the general pattern. In such a case we need to find ways to adjust the model to reduce the gap and improve the performance on the validation set.

In [ ]:
viz_cv_results(cv_results, show_table=False, plot_confidence=False, plot_fit_time=False)

<div class="alert alert-success">


<h3>Questions</h3>

Let's explore the plot above. 

__Q1.__ Do we observve overfitting i.e. a large gap between the training and validation curves?

__Q2.__ What happens when C is very small and when it is very large?


</div>

#### Give your answer here:

1.    




2.











### Model evaluation

In [ ]:
# Print classification report for test set
classification_report_(df_test, model)

# Print predictions on test set
plot_confusion_matrix(df_test, model)

### Get more insights into the model

In the Logistic Regression the coefficients tell us how much each feature contributes to the overall prediction. The larger the absolute value of a coefficient, the more important the corresponding feature is for the model. 

For the numerical features and the text features we always have positive feature values. Thus positive coefficients contribute more to the `spam` class while negative coefficients contribute more to the `non-spam` class.

For the embedding features we have both positive and negative feature values. Thus we need to look at the contributions (feature values times coefficients) to judge the impact on the overall prediction. This will help us understand the model's behavior better and identify which features are driving the predictions for particular samples.

In [ ]:
visualize_coefficients(model, df_train, n_top_features=5)

### How sure was the model of its predictions?
The Logistic Regression model can return the probabilities of each class for each sample. The probabilities are between 0 and 1, and the sum of the probabilities for each sample is 1. This allows us to assess the confidence of the model's predictions.

Below we plot the probabilities of the spam class and colour them by their actual class. Low probabilities (close to 0) indicate that the model is very sure that the sample is not spam, while high probabilities (close to 1) indicate that the model is very sure that the sample is spam.

In [ ]:
plot_prediction_certainties(df_test, model, log_scale=False)

<div class="alert alert-success">


<h3>Questions</h3>

Let's explore the plot above. 

__Q1.__ When the model misclassified a sample, is it usually very sure of its prediction, or kind of doubtful?

Careful with interpretation when the top plot using a log-scale. This means that the values are not evenly spaced. You can change the setting.

</div>

#### Give your answer here:

1.    















### Error analysis :: Where does our model fail?

We will now analyze the misclassified mails in order to get some insights on where the model failed to make correct predictions. The *error_analysis* function below will show us the top features responsible for the model making a decision of prediction whether the mail is spam or non-spam.

In [ ]:
%run modeling_tools.py

In [ ]:
error_analysis(df_test, model, doc_nbr=10, n_top_coeff=5, color_by_coeff_sign=True)

<div class="alert alert-info">


# NOW IT'S YOUR TURN

We have copied the above code blocks again below. You can use them to build your own spam detector now. 

There are a number of things you can adapt:

### Change the feature space

We have loaded 4 feature spaces at the start of the notebook. Simply replace `num_features_df` with `text_features`, `num_text_features`, or `embedding_features` in the code below to use a different feature space.

Warning: The feature spaces using text features are quite slow (45 mins) and will take quite a while to run the fine-tuning with cross-validation.  
The pre-computed output of grid search with cross-validation can be loaded with the following code `pd.read_csv("text_log_reg_cv_results.csv")`
    
You can retrain the model using the `fit_model`-function with `C` set to the best `C`-value from `cv_results`.
    
### Change the metric used for fine-tuning

You can change the scoring function inside `fit_log_reg_model(df_train)`.  

The current default value is `None`, which means that the model will use the default scoring function for the Logistic Regression model. This is the accuracy score. 

However, you can also change the scoring function to `"precision"`, `"recall"`, or `"f1"` and check how the results change. More options are given [here](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

What happens to the confusion matrix as you vary the metric?

### Explore other settings

You will likely have to change some of the other parameters in the visualisations, etc. to make them more interpretable.

</div>

In [ ]:
# To make things easier:
# Change your settings here and then run the cell below

feature_space = num_features_df
# options are: num_features_df, text_features_df, num_text_features_df, embeddings_df


C = 1
# options to try are: 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000


scoring = None
# options include: 'accuracy', 'f1', 'precision', 'recall'

In [ ]:
# Train/test splitting
df_train, df_test = train_test_split_(feature_space)

# Fit model on the train data
model = fit_model(df_train, C=C)

# Print classification report for test set
classification_report_(df_test, model)

# Print predictions on test set
plot_confusion_matrix(df_test, model)

In [ ]:
# Train/test splitting
df_train, df_test = train_test_split_(feature_space)
# text_features_df takes 45mins

# Fit model on the train data
model, cv_results = fit_log_reg_model(df_train, scoring=scoring)

#### Tracking overfitting

In [ ]:
viz_cv_results(cv_results, show_table=False, plot_confidence=False, plot_fit_time=False)

### Model evaluation

In [ ]:
# Print classification report for test set
classification_report_(df_test, model)

# Print predictions on test set
plot_confusion_matrix(df_test, model)

### Get more insights into the model

In [ ]:
visualize_coefficients(model, df_train, n_top_features=10)

### How sure was the model of its predictions?


In [ ]:
plot_prediction_certainties(df_test, model, log_scale=True)

### Error analysis :: Where does our model fail?

In [ ]:
error_analysis(df_test, model, doc_nbr=10, n_top_coeff=15, color_by_coeff_sign=True)